# Visualize test stimuli

In [ ]:
from starling_rhythm.utils.paths import PROCESSED_DIR, ensure_dir
import pandas as pd
import numpy as np
from scipy.io.wavfile import read, write
from starling_rhythm.utils.audio import extract_amp_env
import matplotlib.pyplot as plt

In [ ]:
bID = 's_b1555_22'

In [ ]:
MF_PATH = PROCESSED_DIR / bID / 'salvage_tmf.pickle'

In [ ]:
stims = pd.read_pickle(MF_PATH)

In [ ]:
stims

In [ ]:
pair_index = []
for i, row in stims.iterrows():
    pair_index.append(str(row.address).split('/')[-1].split('_')[-2])
    
stims['pair_index'] = pair_index

In [ ]:
stims_0 = stims[stims.pair_index == '0']

In [ ]:
stims_1 = stims[stims.pair_index == '1']

In [ ]:
stims_0['rank'] = stims_0['mean_tMF'].rank(ascending = False)
stims_1['rank'] = stims_1['mean_tMF'].rank(ascending = False)

## Additional trimming

In [ ]:
stims_0 = stims_0[(stims_0.mean_tMF < 92.718680) & (stims_0.mean_tMF > 37.848900)]
stims_1 = stims_1[(stims_1.mean_tMF < 60.389137) & (stims_1.mean_tMF > 8.363697)]

In [ ]:
def trimmer(stim_df):
    target_array = np.linspace(stim_df.mean_tMF.min(), stim_df.mean_tMF.max(), 128)
    column_values = stim_df.mean_tMF.values
    closest_indices = []
    used_indices = []
    for i in range(len(target_array)):
        # Calculate the absolute difference between each element in the column and the target array
        abs_diff = np.abs(column_values - target_array[i])

        # Find the closest index that has not been used
        available_indices = list(set(range(len(column_values))) - set(used_indices))
        closest_idx = available_indices[np.argmin(abs_diff[available_indices])]

        # Add the closest index to the list of closest indices and mark it as used
        closest_indices.append(closest_idx)
        used_indices.append(closest_idx)

    print(closest_indices)
        
    return stim_df.loc[closest_indices]

In [ ]:
stims_0_trimmed = trimmer(stims_0.reset_index(drop = True))

In [ ]:
import seaborn as sns

In [ ]:
sns.histplot(stims_0_trimmed, x = 'mean_tMF')

In [ ]:
sns.histplot(stims_0_trimmed, x = 'rank')

## visualize

In [ ]:
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

In [ ]:
## MF

fig, axs = plt.subplots(len(stims_0), sharex = True, figsize=(18, 36), dpi=300)

for i, row in tqdm(stims_0.iterrows()):
    axs[i].title.set_text(str(row.pair_index) + '_' + str(row.mean_tMF) + '_' + str(row['rank']))
    sr, data = read(row.address)
    x = axs[i].specgram(data, Fs = sr, cmap = 'twilight')

In [ ]:
## MF

fig, axs = plt.subplots(len(stims_1), sharex = True, figsize=(18, 36), dpi=300)

for i, row in tqdm(stims_1.iterrows()):
    axs[i].title.set_text(str(row.pair_index) + '_' + str(row.mean_tMF) + '_' + str(row['rank']))
    sr, data = read(row.address)
    x = axs[i].specgram(data, Fs = sr, cmap = 'twilight')